In [11]:
import sys
sys.path.append('.')

import import_ipynb
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from DocumentLoader import DocumentLoader
from DocumentTextSplitterManager import DocumentTextSplitterManager
from EmbeddingManager import EmbeddingManager

In [12]:
documentloader = DocumentLoader()
loaded_documents = documentloader.load_directory(directory_path="data", glob_pattern="**/*.pdf")
print(loaded_documents)


Loading documents....


INFO:DocumentLoader:Total documents loaded 8


loaded documents
[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Everstorm  Outfitters    RETURN  &  EXCHANGE  POLICY    Document  ROX-2025-05   Easy-Fit  Promise    If  your  gear  doesn’t  fit  or  just  isn’t  your  vibe,  send  it  back  within  **30  days**  of  delivery  for  a  refund  or  free  size  exchange.   Eligibility  Checklist    ●  Unworn,  unwashed,  no  odors,  tags  attached    ●  Original  shoe  box  (footwear)  placed  inside  outer  carton    ●  Electronics  (power-banks,  headlamps)  unopened  unless  faulty   How  to  Start    ●  Visit  everstorm.example/returns  →  enter  order  #  and  email.    ●  Select  “Refund”  or  “Exchange.”    ●  Print  prepaid  label;  pack  securely.  Multiple  items  can  share  one  box.   Instant  Exchange  Hol

INFO:DocumentLoader:Total documents loaded 8


[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Everstorm  Outfitters    RETURN  &  EXCHANGE  POLICY    Document  ROX-2025-05   Easy-Fit  Promise    If  your  gear  doesn’t  fit  or  just  isn’t  your  vibe,  send  it  back  within  **30  days**  of  delivery  for  a  refund  or  free  size  exchange.   Eligibility  Checklist    ●  Unworn,  unwashed,  no  odors,  tags  attached    ●  Original  shoe  box  (footwear)  placed  inside  outer  carton    ●  Electronics  (power-banks,  headlamps)  unopened  unless  faulty   How  to  Start    ●  Visit  everstorm.example/returns  →  enter  order  #  and  email.    ●  Select  “Refund”  or  “Exchange.”    ●  Print  prepaid  label;  pack  securely.  Multiple  items  can  share  one  box.   Instant  Exchange  Hold    We  place  a

In [13]:
textSplitterManager = DocumentTextSplitterManager()
splitter = textSplitterManager.get_recursive_splitter()
chunks = splitter.split_documents(loaded_documents)
print(chunks)

INFO:DocumentTextSplitterManager:Created recursive character spliersize:100, overlap: 20


[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Everstorm  Outfitters    RETURN  &  EXCHANGE  POLICY    Document  ROX-2025-05   Easy-Fit  Promise'), Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Promise    If  your  gear  doesn’t  fit  or  just  isn’t  your  vibe'), Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content=',  send  it  bac

In [14]:
embeddingManager = EmbeddingManager()
embeddings = embeddingManager.embeddings


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-small
INFO:EmbeddingManager:Initialized sentenceTransformer embeddings


In [15]:
vectorDb = FAISS.from_documents(chunks, embeddings)
query ="what Footwear Conversion do you make use of"

vectorDb.similarity_search(query,k=8)

retriever = vectorDb.as_retriever(search_kwargs= {"k": 8})



In [16]:
llm = Ollama(model="gemma3:1b", temperature="0.5")
text = "Hi my name is ibrahim?"
response = llm.invoke(text)
print(response)

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x317c07050>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [18]:
SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
"""

In [21]:
vectorDb = FAISS.from_documents(chunks, embeddings)

retriever = vectorDb.as_retriever(search_kwargs= {"k": 8})

llm = Ollama(
    model="gemma3:1b",
    temperature=0.5
)

prompt = PromptTemplate(input_variables=["context","question"],template=SYSTEM_TEMPLATE)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
chain = ConversationalRetrievalChain.from_llm(llm,
                                     retriever, 
                                     combine_docs_chain_kwargs={"prompt":prompt},
                                     return_source_documents=True
                                     )



In [25]:
test_questions = [
    "If i'm not happy with my purchase, what is your refund policy and how do i start a return?",
    "How long will delivery take for a standard order, and where can i track my package once it ships",
    "What's the quickest way to contact your support team, and what are your operating hours?"
]

chat_history = []

for q in test_questions:
    result = chain({"question": q, "chat_history": chat_history})
    

    chat_history.append((q, result["answer"]))
    
    print(f"\nQ: {q}\nA: {result['answer'][:350]}...")



Q: If i'm not happy with my purchase, what is your refund policy and how do i start a return?
A: If you’re not happy with your purchase, you can initiate a refund. We issue refunds the same day your return is scanned. Send it back within 30 days of delivery for a refund or free size exchange. Return Window Exceptions:
*   Holiday gift purchases made between November 1st and December 31st are subject to return.
*   Refund timeline: Warehouse re...

Q: How long will delivery take for a standard order, and where can i track my package once it ships
A: Delivery times may take up to 12 hours to appear after the parcel is scanned at [source: U.S] warehouse time. You can track your package once it ships at [source: logistics@everstorm.example] +1 (406)....

Q: What's the quickest way to contact your support team, and what are your operating hours?
A: At checkout, the rate is locked for 15 minutes of order placement. You can contact them at billing@everstorm.example or call 08:00–18:00 MT at 